In [1]:
import os
import pandas as pd
import numpy as np
import sys
import operator
from copy import copy

In [2]:
demand_file_name = os.path.join('..', 'data', 'input_files_MckeesRocks_SPC', 'MNM_input_demand_new')
# output_file_name = os.path.join('..', 'data', 'input_files_MckeesRocks_SPC', 'MNM_input_demand')

## Read all demand

In [3]:
demand_map_car = {}
demand_map_truck = {}
with open(demand_file_name, 'r') as infile:
    for line in infile.readlines():
        if line[0] != '#':
            demands = line.rstrip('\n').split(' ')
            num_int = (len(demands) - 2)//2
            demand_map_car[demands[0]+' '+demands[1]] = np.array([float(e) for e in demands[2:26]])
            demand_map_truck[demands[0]+' '+demands[1]] = np.array([float(e) for e in demands[26:50]])
#             print(demand_map_car)
#             print(demand_map_truck)
#             break

## Read all paths & store all OD for revise & change OD demands

In [18]:
ids = ['4150', '7186', '3858']
# change OD by dividing the rations below
ratios_car = [[3.0399271291938668*4, 3.2335958005249346*4, 3.5325260490002814*4, 3.532781667727562*4, 3.2814589665653497*4, 2.817129006194452*4],
             [1.5105624619664435*3, 1.189433962264151*3, 1.3883730003901678*3, 2.297978745571994*3, 2.0725030826140567*3, 2.9351501387837495*3],
             [0.26284074605451935/1.5, 0.2669897565811213/1.5, 0.8502002670226969/1.5, 0.6784572767952269/1.5, 0.8605077204920178, 1.0979115812313534],
             [0.6872696342500709, 0.9698220273407274, 2.000603318250377, 0.97165991902834, 1.612422360248447, 2.0677872692201706]]
ratios_truck = [[2.064981949458484*4, 1.9851411589895989*4, 2.0500736377025035*4, 1.5840597758405977*4, 1.2682926829268293*4, 1.6016177957532862*4],
               [1.2670807453416149*3, 1.1888888888888889*3, 1.1428571428571428*3, 1.7448071216617211*3, 1.5528255528255528*3, 2.795847750865052*3],
               [0.2626865671641791/1.5, 0.366412213740458/1.5, 0.8588469184890656/1.5, 0.44274809160305345/1.5, 0.3151969981238274/1.5, 0.6408268733850129/1.5],
               [0.8602150537634409, 0.8605577689243028, 1.9259259259259258, 0.7639257294429708, 1.0961538461538463, 1.3436692506459949]]

path_file_name = os.path.join('..', 'data', 'input_files_MckeesRocks_SPC', 'path_table')

for j in range(3):
    _id = ids[j]
    print(_id)
    od_car = []
    od_truck = []
    with open(path_file_name, 'r') as infile:
        for line in infile.readlines():
            links = line.rstrip('\n').split(' ')
            O = links[0]
            D = links[-1]
            all_link = links[1:-1]
            if _id in all_link:
#                 print('2' + O[1:] + ' ' + '2' + D[1:])
                od_car.append('2' + O[1:] + ' ' + '2' + D[1:])
                od_truck.append('2' + O[1:] + ' ' + '2' + D[1:])
    for key in od_car:
        if key in demand_map_car:
#             print(demand_map_car[key])
            for i in range(6):     
                demand_map_car[key][i*4:i*4+3] /= ratios_car[j][i]
#             print(demand_map_car[key])
#             break
    for key in od_truck:
        if key in demand_map_truck:
            for i in range(6):     
                demand_map_truck[key][i*4:i*4+3] /= ratios_truck[j][i]

4150
7186
3858


## Output all modified demand

In [19]:
demand_str = '# Origin_ID Destination_ID <demand by interval> <truck demand by interval>\n'

for key in demand_map_car:
#     print(key)
#     print(demand_map_car[key])
#     print(demand_map_truck[key])
    _cars = [str(e) for e in demand_map_car[key]]
    _trucks = [str(e) for e in demand_map_truck[key]]
    _all = [key] + _cars + _trucks
#     print(_all)
#     break
    demand_str += ' '.join(_all) + '\n'


# f = open(output_file_name, 'w')
# f.write(demand_str)
# f.close()

## Opening year case & Full buildout year case

In [20]:
output_file_name_2 = os.path.join('..', 'data', 'input_files_MckeesRocks_SPC2', 'MNM_input_demand')
output_file_name_3 = os.path.join('..', 'data', 'input_files_MckeesRocks_SPC3', 'MNM_input_demand')

In [22]:
demand_str = '# Origin_ID Destination_ID <demand by interval> <truck demand by interval>\n'

demand_map_car2 = copy(demand_map_car)
demand_map_truck2 = copy(demand_map_truck)

truck_in2 = 157.0
truck_out2 = 56.0
if '200010 250123' in demand_map_truck2:
    demand_map_truck2['200010 250123'] += truck_in2 / 2 / 24
else:
    demand_map_truck2['200010 250123'] = truck_in2 / 2 / 24
if '200247 250123' in demand_map_truck2:
    demand_map_truck2['200247 250123'] += truck_in2 / 2 / 24
else:
    demand_map_truck2['200247 250123'] = truck_in2 / 2 / 24   
if '200123 250010' in demand_map_truck2:
    demand_map_truck2['200123 250010'] += truck_out2 / 2 / 24
else:
    demand_map_truck2['200123 250010'] = truck_out2 / 2 / 24
if '200123 250247' in demand_map_truck2:
    demand_map_truck2['200123 250247'] += truck_out2 / 2 / 24
else:
    demand_map_truck2['200123 250247'] = truck_out2 / 2 / 24
    
for key in demand_map_car2:
    _cars = [str(e) for e in demand_map_car2[key]]
    _trucks = [str(e) for e in demand_map_truck2[key]]
    _all = [key] + _cars + _trucks
    demand_str += ' '.join(_all) + '\n'

f = open(output_file_name_2, 'w')
f.write(demand_str)
f.close()

In [23]:
demand_str = '# Origin_ID Destination_ID <demand by interval> <truck demand by interval>\n'

demand_map_car3 = copy(demand_map_car)
demand_map_truck3 = copy(demand_map_truck)

car_in3 = 19.0
# car_out3 = 0.0
if '200010 250123' in demand_map_car3:
    demand_map_car3['200010 250123'] += car_in3 / 4 / 24
else:
    demand_map_car3['200010 250123'] = car_in3 / 4 / 24
if '200247 250123' in demand_map_car3:
    demand_map_car3['200247 250123'] += car_in3 / 4 / 24
else:
    demand_map_car3['200247 250123'] = car_in3 / 4 / 24
if '200251 250123' in demand_map_car3:
    demand_map_car3['200251 250123'] += car_in3 / 4 / 24
else:
    demand_map_car3['200251 250123'] = car_in3 / 4 / 24
if '200063 250123' in demand_map_car3:
    demand_map_car3['200063 250123'] += car_in3 / 4 / 24
else:
    demand_map_car3['200063 250123'] = car_in3 / 4 / 24
    
truck_in3 = 272.0
truck_out3 = 98.0
if '200010 250123' in demand_map_truck3:
    demand_map_truck3['200010 250123'] += truck_in3 / 2 / 24
else:
    demand_map_truck3['200010 250123'] = truck_in3 / 2 / 24
if '200247 250123' in demand_map_truck3:
    demand_map_truck3['200247 250123'] += truck_in3 / 2 / 24
else:
    demand_map_truck3['200247 250123'] = truck_in3 / 2 / 24    
if '200123 250010' in demand_map_truck3:
    demand_map_truck3['200123 250010'] += truck_out3 / 2 / 24
else:
    demand_map_truck3['200123 250010'] = truck_out3 / 2 / 24
if '200123 250247' in demand_map_truck3:
    demand_map_truck3['200123 250247'] += truck_out3 / 2 / 24
else:
    demand_map_truck3['200123 250247'] = truck_out3 / 2 / 24
    
for key in demand_map_car3:
    _cars = [str(e) for e in demand_map_car3[key]]
    _trucks = [str(e) for e in demand_map_truck3[key]]
    _all = [key] + _cars + _trucks
    demand_str += ' '.join(_all) + '\n'

f = open(output_file_name_3, 'w')
f.write(demand_str)
f.close()

# Scenario 0

In [6]:
output_s0 = os.path.join('..', 'data', 'input_files_MckeesRocks_SPC_scenario_0', 'MNM_input_demand')

demand_str = '# Origin_ID Destination_ID <demand by interval> <truck demand by interval>\n'

demand_map_car = copy(demand_map_car)
demand_map_truck = copy(demand_map_truck)

car_in = 19.0
# car_out = 0.0
if '200120 250123' in demand_map_car:
    demand_map_car['200120 250123'] += car_in / 3 / 24
else:
    demand_map_car['200120 250123'] = car_in / 3 / 24
if '200194 250123' in demand_map_car:
    demand_map_car['200194 250123'] += car_in / 3 / 24
else:
    demand_map_car['200194 250123'] = car_in / 3 / 24
if '200275 250123' in demand_map_car:
    demand_map_car['200275 250123'] += car_in / 3 / 24
else:
    demand_map_car['200275 250123'] = car_in / 3 / 24

    
truck_in = 272.0
truck_out = 98.0
if '200120 250123' in demand_map_truck:
    demand_map_truck['200120 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200120 250123'] = truck_in / 3 / 24
if '200194 250123' in demand_map_truck:
    demand_map_truck['200194 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200194 250123'] = truck_in / 3 / 24    
if '200275 250123' in demand_map_truck:
    demand_map_truck['200275 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200275 250123'] = truck_in / 3 / 24 
    
if '200123 250120' in demand_map_truck:
    demand_map_truck['200123 250120'] += truck_out / 3 / 24
else:
    demand_map_truck['200123 250120'] = truck_out / 3 / 24
if '200123 250194' in demand_map_truck:
    demand_map_truck['200123 250194'] += truck_out / 3 / 24
else:
    demand_map_truck['200123 250194'] = truck_out / 3 / 24
if '200123 250275' in demand_map_truck:
    demand_map_truck['200123 250275'] += truck_out / 3 / 24
else:
    demand_map_truck['200123 250275'] = truck_out / 3 / 24
    
for key in demand_map_car:
    _cars = [str(e) for e in demand_map_car[key]]
    _trucks = [str(e) for e in demand_map_truck[key]]
    _all = [key] + _cars + _trucks
    demand_str += ' '.join(_all) + '\n'

f = open(output_s0, 'w')
f.write(demand_str)
f.close()

# Scenario 3

In [4]:
output_s3 = os.path.join('..', 'data', 'input_files_MckeesRocks_SPC_scenario_3', 'MNM_input_demand')

demand_str = '# Origin_ID Destination_ID <demand by interval> <truck demand by interval>\n'

demand_map_car = copy(demand_map_car)
demand_map_truck = copy(demand_map_truck)

car_in = 4.0
# car_out = 0.0
if '200120 250123' in demand_map_car:
    demand_map_car['200120 250123'] += car_in / 3 / 24
else:
    demand_map_car['200120 250123'] = car_in / 3 / 24
if '200194 250123' in demand_map_car:
    demand_map_car['200194 250123'] += car_in / 3 / 24
else:
    demand_map_car['200194 250123'] = car_in / 3 / 24
if '200275 250123' in demand_map_car:
    demand_map_car['200275 250123'] += car_in / 3 / 24
else:
    demand_map_car['200275 250123'] = car_in / 3 / 24

    
truck_in = 135.0
truck_out = 42.0
if '200120 250123' in demand_map_truck:
    demand_map_truck['200120 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200120 250123'] = truck_in / 3 / 24
if '200194 250123' in demand_map_truck:
    demand_map_truck['200194 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200194 250123'] = truck_in / 3 / 24    
if '200275 250123' in demand_map_truck:
    demand_map_truck['200275 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200275 250123'] = truck_in / 3 / 24 
    
if '200123 250120' in demand_map_truck:
    demand_map_truck['200123 250120'] += truck_out / 3 / 24
else:
    demand_map_truck['200123 250120'] = truck_out / 3 / 24
if '200123 250194' in demand_map_truck:
    demand_map_truck['200123 250194'] += truck_out / 3 / 24
else:
    demand_map_truck['200123 250194'] = truck_out / 3 / 24
if '200123 250275' in demand_map_truck:
    demand_map_truck['200123 250275'] += truck_out / 3 / 24
else:
    demand_map_truck['200123 250275'] = truck_out / 3 / 24
    
for key in demand_map_car:
    _cars = [str(e) for e in demand_map_car[key]]
    _trucks = [str(e) for e in demand_map_truck[key]]
    _all = [key] + _cars + _trucks
    demand_str += ' '.join(_all) + '\n'

f = open(output_s3, 'w')
f.write(demand_str)
f.close()

# Scenario 4

In [4]:
output_s4 = os.path.join('..', 'data', 'input_files_MckeesRocks_SPC_scenario_4', 'MNM_input_demand')

demand_str = '# Origin_ID Destination_ID <demand by interval> <truck demand by interval>\n'

demand_map_car = copy(demand_map_car)
demand_map_truck = copy(demand_map_truck)

car_in = 19.0
# car_out = 0.0
if '200120 250123' in demand_map_car:
    demand_map_car['200120 250123'] += car_in / 3 / 24
else:
    demand_map_car['200120 250123'] = car_in / 3 / 24
if '200194 250123' in demand_map_car:
    demand_map_car['200194 250123'] += car_in / 3 / 24
else:
    demand_map_car['200194 250123'] = car_in / 3 / 24
if '200275 250123' in demand_map_car:
    demand_map_car['200275 250123'] += car_in / 3 / 24
else:
    demand_map_car['200275 250123'] = car_in / 3 / 24

    
truck_in = 272.0
truck_out = 98.0
if '200120 250123' in demand_map_truck:
    demand_map_truck['200120 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200120 250123'] = truck_in / 3 / 24
if '200194 250123' in demand_map_truck:
    demand_map_truck['200194 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200194 250123'] = truck_in / 3 / 24    
if '200275 250123' in demand_map_truck:
    demand_map_truck['200275 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200275 250123'] = truck_in / 3 / 24 
    

if '200123 250275' in demand_map_truck:
    demand_map_truck['200123 250275'] += truck_out / 24
else:
    demand_map_truck['200123 250275'] = truck_out / 24
    
for key in demand_map_car:
    _cars = [str(e) for e in demand_map_car[key]]
    _trucks = [str(e) for e in demand_map_truck[key]]
    _all = [key] + _cars + _trucks
    demand_str += ' '.join(_all) + '\n'

f = open(output_s4, 'w')
f.write(demand_str)
f.close()

# Scenario 6

In [8]:
output_s6 = os.path.join('..', 'data', 'input_files_MckeesRocks_SPC_scenario_6', 'MNM_input_demand')

demand_str = '# Origin_ID Destination_ID <demand by interval> <truck demand by interval>\n'

demand_map_car = copy(demand_map_car)
demand_map_truck = copy(demand_map_truck)

car_in = 19.0
# car_out = 0.0
if '200120 250123' in demand_map_car:
    demand_map_car['200120 250123'] += car_in / 3 / 24
else:
    demand_map_car['200120 250123'] = car_in / 3 / 24
if '200194 250123' in demand_map_car:
    demand_map_car['200194 250123'] += car_in / 3 / 24
else:
    demand_map_car['200194 250123'] = car_in / 3 / 24
if '200275 250123' in demand_map_car:
    demand_map_car['200275 250123'] += car_in / 3 / 24
else:
    demand_map_car['200275 250123'] = car_in / 3 / 24

    
truck_in = 272.0
truck_out = 98.0
if '200120 250123' in demand_map_truck:
    demand_map_truck['200120 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200120 250123'] = truck_in / 3 / 24
if '200194 250123' in demand_map_truck:
    demand_map_truck['200194 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200194 250123'] = truck_in / 3 / 24    
if '200275 250123' in demand_map_truck:
    demand_map_truck['200275 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200275 250123'] = truck_in / 3 / 24 
    
if '200123 250120' in demand_map_truck:
    demand_map_truck['200123 250120'] += truck_out / 3 / 24
else:
    demand_map_truck['200123 250120'] = truck_out / 3 / 24
if '200123 250194' in demand_map_truck:
    demand_map_truck['200123 250194'] += truck_out / 3 / 24
else:
    demand_map_truck['200123 250194'] = truck_out / 3 / 24
if '200123 250275' in demand_map_truck:
    demand_map_truck['200123 250275'] += truck_out / 3 / 24
else:
    demand_map_truck['200123 250275'] = truck_out / 3 / 24
    
for key in demand_map_car:
    if key.split(' ')[1] == '250123' or key.split(' ')[1] == '250251' or key.split(' ')[1] == '250063' \
       or key.split(' ')[1] == '250010' or key.split(' ')[1] == '250247' or key.split(' ')[0] == '200123' \
       or key.split(' ')[0] == '200251' or key.split(' ')[0] == '200063' or key.split(' ')[0] == '200010' \
       or key.split(' ')[0] == '200247':
        demand_map_car[key] = demand_map_car[key] * 1.2    
    
for key in demand_map_car:
    _cars = [str(e) for e in demand_map_car[key]]
    _trucks = [str(e) for e in demand_map_truck[key]]
    _all = [key] + _cars + _trucks
    demand_str += ' '.join(_all) + '\n'

f = open(output_s6, 'w')
f.write(demand_str)
f.close()

# Scenario 7

In [4]:
output_s7 = os.path.join('..', 'data', 'input_files_MckeesRocks_SPC_scenario_7', 'MNM_input_demand')

demand_str = '# Origin_ID Destination_ID <demand by interval> <truck demand by interval>\n'

demand_map_car = copy(demand_map_car)
demand_map_truck = copy(demand_map_truck)

car_in = 19.0
# car_out = 0.0
if '200120 250123' in demand_map_car:
    demand_map_car['200120 250123'] += car_in / 3 / 24
else:
    demand_map_car['200120 250123'] = car_in / 3 / 24
if '200194 250123' in demand_map_car:
    demand_map_car['200194 250123'] += car_in / 3 / 24
else:
    demand_map_car['200194 250123'] = car_in / 3 / 24
if '200275 250123' in demand_map_car:
    demand_map_car['200275 250123'] += car_in / 3 / 24
else:
    demand_map_car['200275 250123'] = car_in / 3 / 24

    
truck_in = 272.0
truck_out = 98.0
if '200120 250123' in demand_map_truck:
    demand_map_truck['200120 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200120 250123'] = truck_in / 3 / 24
if '200194 250123' in demand_map_truck:
    demand_map_truck['200194 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200194 250123'] = truck_in / 3 / 24    
if '200275 250123' in demand_map_truck:
    demand_map_truck['200275 250123'] += truck_in / 3 / 24
else:
    demand_map_truck['200275 250123'] = truck_in / 3 / 24 
    
if '200123 250120' in demand_map_truck:
    demand_map_truck['200123 250120'] += truck_out / 3 / 24
else:
    demand_map_truck['200123 250120'] = truck_out / 3 / 24
if '200123 250194' in demand_map_truck:
    demand_map_truck['200123 250194'] += truck_out / 3 / 24
else:
    demand_map_truck['200123 250194'] = truck_out / 3 / 24
if '200123 250275' in demand_map_truck:
    demand_map_truck['200123 250275'] += truck_out / 3 / 24
else:
    demand_map_truck['200123 250275'] = truck_out / 3 / 24
    
for key in demand_map_truck:
    if key.split(' ')[1] == '250123' or key.split(' ')[1] == '250251' or key.split(' ')[1] == '250063' \
       or key.split(' ')[1] == '250010' or key.split(' ')[1] == '250247' or key.split(' ')[0] == '200123' \
       or key.split(' ')[0] == '200251' or key.split(' ')[0] == '200063' or key.split(' ')[0] == '200010' \
       or key.split(' ')[0] == '200247':
        demand_map_truck[key] = demand_map_truck[key] * 1.1
    
for key in demand_map_car:
    _cars = [str(e) for e in demand_map_car[key]]
    _trucks = [str(e) for e in demand_map_truck[key]]
    _all = [key] + _cars + _trucks
    demand_str += ' '.join(_all) + '\n'

f = open(output_s7, 'w')
f.write(demand_str)
f.close()